## Parsa ordklasser
Den här anteckningsboken tar rader med ord som har taggats på olika sätt och kollar vilka ord som tillhör flera ordklasser.

Raderna ser typiskt ut så här:

```<w posset="|VB|" pos="VB" msd="VB.PRS.AKT" lemma="|framgå|" lex="|framgå..vb.1|" saldo="|framgå..1|" prefix="|fram..ab.1|" suffix="|gå..vb.1|" ref="12" dephead="10" deprel="ET">framgår</w>```

Det jag är ute efter är själva ordet, ordklassen (och saldo-referensen).

Filen är egentligen en xml-fil, men eftersom alla orden representeras som ett ord per rad fuskar jag och läser rad för rad, och tar bort alla som inte börjar på "<w", som är tecknet på att det är ett ord (och inte en mening etc).

In [315]:
import pickle

In [139]:
import re
import numpy as np

In [158]:
def parse(line):
    word = re.search('(?<=>)(.*)(?=<)', line).group(0) #Hitta det som är mellan ett > och ett <
    pos = re.search('(?<=pos=)(.*\s)(?=m)', line).group(0).strip(' ').strip('"') #Hitta det som är mellan"pos" och följande "m"
    saldo = re.search('(?<=saldo=)(.*\s)(?=p)', line).group(0).strip(' ').strip('"')
    #print (word, '\n', pos, '\n', saldo, '\n')
    return np.array([word,pos,saldo]) 

In [343]:
x= np.empty((1,3)) 
with open('moderntdv_1000.xml') as f:
    for line in f:
        if line.find('<w ')>-1:
            tags = parse(line)
            x = np.vstack([x,tags])
        
with open('pos_tagged.pickle', 'wb') as f:
    pickle.dump(x,f)
 #       str.find(str, beg=0, end=len(string))

In [344]:
x

array([['0.0', '0.0', '0.0'],
       ['(', 'PAD', '|'],
       ['Jfr', 'VB', '|jämföra..1|'],
       ..., 
       ['arbetet', 'NN', '|arbete..1|arbete..2|'],
       ['själv', 'JJ', '|själv..1|själv..2|själv..3|'],
       ['.', 'MAD', '|']],
      dtype='<U104')

In [345]:
wordlist = []
while len(x)>1:
    x=x[1:] #Ta bort första raden
    #x[0]

    if x[0][0].isalpha():
        #print(x[0][0])
        indexar = np.array(np.where(x[:,0] == x[0][0])) #Radnumret för alla förekomster av ordet i texten

        ordklasser = x[indexar] #Alla rader där ordet förekommer, som np.array

    olika = np.unique(ordklasser[0][:,1]) #Listan över unika ordklasser för det ordet

    
    if len (olika)>1 and not x[0][0] in wordlist:
        wordlist.append(x[0][0])
        print (x[0][0])
        print (olika)
        #print (indexar)
        print ('------------------------------------------------\n')
        np.delete(x,indexar[1:],axis=0)

om
['PP' 'SN']
------------------------------------------------

som
['HP' 'KN']
------------------------------------------------

den
['DT' 'PN']
------------------------------------------------

d
['DT' 'NN']
------------------------------------------------

21
['DT' 'NN']
------------------------------------------------

m
['NN" msd="NN.AN" lemma="|m|m' 'NN" msd="NN.AN" lemma="|m|m m:25|m']
------------------------------------------------

.
['NN" msd="NN.AN" lemma="|m|m' 'NN" msd="NN.AN" lemma="|m|m m:25|m']
------------------------------------------------

7
['DT' 'NN']
------------------------------------------------

till
['PL' 'PP']
------------------------------------------------

L.W
['PL' 'PP']
------------------------------------------------

D
['NN' 'PM']
------------------------------------------------

för
['AB' 'PP']
------------------------------------------------

208
['AB' 'PP']
------------------------------------------------

000
['AB' 'PP']
-----------------------

In [346]:
wordlist

['om',
 'som',
 'den',
 'd',
 '21',
 'm',
 '.',
 '7',
 'till',
 'L.W',
 'D',
 'för',
 '208',
 '000',
 'stadsägan',
 ',',
 '404,7',
 '7027',
 'det',
 '5,40',
 'andra',
 'att',
 '36',
 'planerat',
 'bestritt',
 'vitsordat',
 '8']